In [ ]:
from time import perf_counter
start_total = perf_counter()

In [ ]:
%%capture
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd
from scipy.integrate import odeint
!pip install -U kaleido 
!pip install kora -q
import kora.install.orca
import os
if not os.path.exists("images"):
    os.mkdir("images")
import networkx as nx
import random
from collections import defaultdict
!pip install kaleido
!pip install plotly>=4.0.0
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get update
!apt-get install xvfb libgtk2.0-0 libgconf-2-4 
margem = dict(l=50,r=50,b=50,t=50,pad=4)

# Deterministico

$$
\begin{align*} 
\frac{dS(t)}{dt} =& -\beta \times  \frac{I(t) }{N}S(t)\\
\frac{dI(t)}{dt} =& \beta \times  \frac{I(t) }{N}S(t) - \gamma I(t)\\
\frac{dR(t)}{dt} =& \gamma I(t)\\
\end{align*}
$$

Cravar em 0.5% I_0

In [ ]:
def deriv(y, t, N, beta, gamma):
    S, I, R = y

    print(S, '', I, '', R)

    dSdt = -beta * S * I / N
    dIdt = beta  * S * I / N - gamma * I
    dRdt = gamma * I

    return dSdt, dIdt, dRdt

In [ ]:
N = 100000
I0, R0 = 500, 0
S0 = N - I0 - R0
beta, gamma = 0.625, 0.2
t = np.linspace(0, 60, 1000)
y0 = S0, I0, R0
ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R = ret.T
type(S)

In [ ]:
N = 100000
I0, R0 = 500, 0
S0 = N - I0 - R0
beta, gamma = 0.625, 0.2
t = np.linspace(0, 60, 1000)
y0 = S0, I0, R0
ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R = ret.T
fig = go.Figure()
fig.add_trace(go.Scatter(x = t,
                         y = S,
                         mode = 'lines',
                         name = 'Susceptível'))
fig.add_trace(go.Scatter(x = t,
                         y = I,
                         mode = 'lines',
                         name = 'Infeccioso'))
fig.add_trace(go.Scatter(x = t,
                         y = R,
                         mode = 'lines',
                         name = 'Recuperado'))
fig.update_layout(title='Modelo SIR Determinístico',
                  xaxis_title='Tempo (dias)',
                  yaxis_title='Número de Indivíduos',
                  width=600,
                  height=300,
                  margin=margem
                  )
fig.show()
fig.write_image("images/DeterministicoPT.svg")
fig = go.Figure()
fig.add_trace(go.Scatter(x = t,
                         y = S,
                         mode = 'lines',
                         name = 'Susceptible'))
fig.add_trace(go.Scatter(x = t,
                         y = I,
                         mode = 'lines',
                         name = 'Infectious'))
fig.add_trace(go.Scatter(x = t,
                         y = R,
                         mode = 'lines',
                         name = 'Recovered'))
fig.update_layout(title='Deterministic SIR Model',
                  xaxis_title='Time (days)',
                  yaxis_title='Individuals number',
                  width=600,
                  height=300,
                  margin=margem
                  )
fig.show()
fig.write_image("images/DeterministicoEN.svg")

# Grafo

In [ ]:
triangle = nx.Graph()
triangle.add_edges_from([("A", "B"), ("A", "C"), ("B", "C")])

nx.draw_spring(triangle, with_labels=True)


# Criando o DF

In [ ]:
N = 100000
t = np.linspace(0, 60, 1000)
size = 3
s = pd.Series(['A','B','C'])
# df = pd.DataFrame({'S': [N - 500, N - 500, N - 500], 'I': [500, 500, 500], 'R': [0,0,0], 'beta': [0.8, 0.625, 0.5], 'gamma': [0.2,0.2,0.2]})
df = pd.DataFrame({'S': [N - 500, N - 500, N - 500], 'I': [500, 500, 500], 'R': [0,0,0], 'beta': [0.625, 0.625, 0.625], 'gamma': [0.1,0.2,0.4], 'N': [N, N, N]})

df.set_index(s, inplace=True)
df

In [ ]:
y = df.loc["A"].S, df.loc["A"].I, df.loc["A"].R

print(y)
result_A = deriv(y, t, N, df.loc["A"].beta, df.loc["A"].gamma)

In [ ]:
result_A

## Separação em lugares

In [ ]:
ret_A = odeint(deriv, y, t, args=(N, df.loc["A"].beta, df.loc["A"].gamma))
S_A, I_A, R_A = ret_A.T
fig = go.Figure()
fig.add_trace(go.Scatter(x = t,
                         y = S_A,
                         mode = 'lines',
                         name = 'Susceptível'))
fig.add_trace(go.Scatter(x = t,
                         y = I_A,
                         mode = 'lines',
                         name = 'Infeccioso'))
fig.add_trace(go.Scatter(x = t,
                         y = R_A,
                         mode = 'lines',
                         name = 'Recuperado'))
fig.update_layout(title='Modelo SIR Determinístico',
                  xaxis_title='Tempo (dias)',
                  yaxis_title='Número de Indivíduos',
                  width=600,
                  height=300,
                  margin=margem
                  )
fig.show()

In [ ]:
ret_B = odeint(deriv, y, t, args=(N, df.loc["B"].beta, df.loc["B"].gamma))
S_B, I_B, R_B = ret_B.T
fig = go.Figure()
fig.add_trace(go.Scatter(x = t,
                         y = S_B,
                         mode = 'lines',
                         name = 'Susceptível'))
fig.add_trace(go.Scatter(x = t,
                         y = I_B,
                         mode = 'lines',
                         name = 'Infeccioso'))
fig.add_trace(go.Scatter(x = t,
                         y = R_B,
                         mode = 'lines',
                         name = 'Recuperado'))
fig.update_layout(title='Modelo SIR Determinístico',
                  xaxis_title='Tempo (dias)',
                  yaxis_title='Número de Indivíduos',
                  width=600,
                  height=300,
                  margin=margem
                  )
fig.show()

In [ ]:
ret_C = odeint(deriv, y, t, args=(N, df.loc["C"].beta, df.loc["C"].gamma))
S_C, I_C, R_C = ret_C.T
fig = go.Figure()
fig.add_trace(go.Scatter(x = t,
                         y = S_C,
                         mode = 'lines',
                         name = 'Susceptível'))
fig.add_trace(go.Scatter(x = t,
                         y = I_C,
                         mode = 'lines',
                         name = 'Infeccioso'))
fig.add_trace(go.Scatter(x = t,
                         y = R_C,
                         mode = 'lines',
                         name = 'Recuperado'))
fig.update_layout(title='Modelo SIR Determinístico',
                  xaxis_title='Tempo (dias)',
                  yaxis_title='Número de Indivíduos',
                  width=600,
                  height=300,
                  margin=margem
                  )
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = t,
                         y = I_A,
                         mode = 'lines',
                         name = 'I_A'))
fig.add_trace(go.Scatter(x = t,
                         y = I_B,
                         mode = 'lines',
                         name = 'I_B'))
fig.add_trace(go.Scatter(x = t,
                         y = I_C,
                         mode = 'lines',
                         name = 'I_C'))
fig.update_layout(title='',
                  xaxis_title='Tempo (dias)',
                  yaxis_title='Número de Indivíduos',
                  width=600,
                  height=300,
                  margin=margem
                  )
fig.show()

# interação entre nós

$$
\begin{align*} 
\frac{dS_i(t)}{dt} =& -\beta \times  \frac{\frac{M_{ii}}{N_i}I_i(t) }{\frac{M_{ii}}{N_i}N_i} \frac{M_{ii}}{N_i}S_i(t)\\
\frac{dI_i(t)}{dt} =& \beta \times \frac{\frac{M_{ii}}{N_i}I_i(t) }{\frac{M_{ii}}{N_i}N_i}\frac{M_{ii}}{N_i}S_i(t) - \gamma I_i(t)\\
\frac{dR_i(t)}{dt} =& \gamma I_i(t)\\
\end{align*}
$$

In [ ]:
N = 100000
t = np.linspace(0, 60, 1000)
size = 3
s = pd.Series(['A','B','C'])
# df = pd.DataFrame({'S': [N - 500, N - 500, N - 500], 'I': [500, 500, 500], 'R': [0,0,0], 'beta': [0.8, 0.625, 0.5], 'gamma': [0.2,0.2,0.2]})
df = pd.DataFrame({'S': [N - 500, N - 500, N - 500], 'I': [500, 500, 500], 'R': [0,0,0], 'beta': [0.625, 0.625, 0.625], 'gamma': [0.1,0.2,0.4], 'N': [N, N, N]})

df.set_index(s, inplace=True)
df

def deriv(df, t, M):
  size = len(df)
  dSdt = []
  dIdt = []
  dRdt = []
  for i in range(size):
    S, I, R, N = df.iloc[i].S, df.iloc[i].I, df.iloc[i].R, df.iloc[i].N
    dSdt.append(-beta * (I / N) * S * (M[i][i]/N))
    dIdt.append(beta  * (I / N) * S * (M[i][i]/N) - gamma * I)
    dRdt.append(gamma * I)
  return dSdt, dIdt, dRdt

In [ ]:
ret_C = odeint(deriv(df), y, t, args=(N, df.loc["C"].beta, df.loc["C"].gamma))
S_C, I_C, R_C = ret_C.T

# Fernando

In [ ]:
# Grafo base
triangle = nx.Graph()
triangle.add_edges_from([("A", "B"), ("A", "C"), ("B", "C")])

nx.draw_spring(triangle, with_labels=True)

In [ ]:
import random
E = triangle.number_of_edges()
print("Numero de vertices ", E)
# Colocando peso nos vertices
w = [random.random() for i in range(E)]
print("Pesos dos vertices ", w)
s = max(w)
print("Peso máximo ", s)
w = [ i/s for i in w ] #normalizing
print("Normalizando pesos", w)
#len(w)
k = 0
for i, j in triangle.edges():
    triangle[i][j]['weight'] = w[k]
    k+=1
import matplotlib.pyplot as plt
edgewidth = [d['weight'] for (u,v,d) in triangle.edges(data=True)]
# layout
#pos = nx.spring_layout(G, iterations=50)
pos = nx.spring_layout(triangle)
labels = {}
for i in list(pos.keys()):
    labels[i] = i
# rendering
plt.figure()#figsize=(40,40)
nx.draw_networkx_nodes(triangle, pos)
nx.draw_networkx_edges(triangle, pos, width=edgewidth, node_size=500)
nx.draw_networkx_labels(triangle, pos, labels)
plt.axis('off')

In [ ]:
def deriv_net_1(y, t, N, beta, gamma, M):
    S, I, R = y[0], y[1], y[2]

    print(f'S = {S}', f'I = {I}', f'R = {R}', f'N = {N}', sep='\n')
    print()

    dSdt = (-beta * (I / N) * S * (M/N))
    dIdt = (-dSdt - gamma * I)
    dRdt = (gamma * I)
    return dSdt, dIdt, dRdt

In [ ]:
N = np.array([33333, 33333, 33333])
I0 = np.array([200, 200, 200])
R0 = np.array([0, 0, 0])
S0 = N - I0 - R0
beta, gamma = 0.625, 0.2
t = np.linspace(0, 120, 10000)
y0 = np.concatenate(([S0], [I0], [R0]),axis=0).T
SIR_A = np.zeros((t.size,3))
SIR_B = np.zeros((t.size,3))
SIR_C = np.zeros((t.size,3))
for i in range(t.size):
    y = y0.copy()
    SIR_A[i][0] = y[0][0].copy()
    SIR_A[i][1] = y[0][1].copy()
    SIR_A[i][2] = y[0][2].copy()
    SIR_B[i][0] = y[1][0].copy()
    SIR_B[i][1] = y[1][1].copy()
    SIR_B[i][2] = y[1][2].copy()
    SIR_C[i][0] = y[1][0].copy()
    SIR_C[i][1] = y[1][1].copy()
    SIR_C[i][2] = y[1][2].copy()
    # dSdt_A, dIdt_A, dRdt_A = deriv_net_1(y[0], t, N, beta, gamma, edgewidth[0])
    # dSdt_B, dIdt_B, dRdt_B = deriv_net_1(y[1], t, N, beta, gamma, edgewidth[1])
    # dSdt_C, dIdt_C, dRdt_C = deriv_net_1(y[2], t, N, beta, gamma, edgewidth[2])
    ret_A = odeint(deriv_net_1, y[0], t[0:2], args=(N[0], beta, gamma, 10000))
    ret_B = odeint(deriv_net_1, y[1], t[0:2], args=(N[1], beta, gamma, 15000))
    ret_C = odeint(deriv_net_1, y[2], t[0:2], args=(N[2], beta, gamma, 20000))
    y0 = [ret_A[-1],ret_B[-1],ret_C[-1]]
'''

fig = go.Figure()
fig.add_trace(go.Scatter(x = t,
                         y = SIR_A.T[0],
                         mode = 'lines',
                         name = 'Susceptível'))
fig.add_trace(go.Scatter(x = t,
                         y = SIR_A.T[1],
                         mode = 'lines',
                         name = 'Infeccioso'))
fig.add_trace(go.Scatter(x = t,
                         y = SIR_A.T[2],
                         mode = 'lines',
                         name = 'Recuperado'))
fig.update_layout(title='Modelo SIR Determinístico - Nó A',
                  xaxis_title='Tempo (dias)',
                  yaxis_title='Número de Indivíduos',
                  width=600,
                  height=300,
                  margin=margem
                  )
fig.show()
#
fig = go.Figure()
fig.add_trace(go.Scatter(x = t,
                         y = SIR_B.T[0],
                         mode = 'lines',
                         name = 'Susceptível'))
fig.add_trace(go.Scatter(x = t,
                         y = SIR_B.T[1],
                         mode = 'lines',
                         name = 'Infeccioso'))
fig.add_trace(go.Scatter(x = t,
                         y = SIR_B.T[2],
                         mode = 'lines',
                         name = 'Recuperado'))
fig.update_layout(title='Modelo SIR Determinístico - Nó B',
                  xaxis_title='Tempo (dias)',
                  yaxis_title='Número de Indivíduos',
                  width=600,
                  height=300,
                  margin=margem
                  )
fig.show()
#
fig = go.Figure()
fig.add_trace(go.Scatter(x = t,
                         y = SIR_C.T[0],
                         mode = 'lines',
                         name = 'Susceptível'))
fig.add_trace(go.Scatter(x = t,
                         y = SIR_C.T[1],
                         mode = 'lines',
                         name = 'Infeccioso'))
fig.add_trace(go.Scatter(x = t,
                         y = SIR_C.T[2],
                         mode = 'lines',
                         name = 'Recuperado'))
fig.update_layout(title='Modelo SIR Determinístico - Nó C',
                  xaxis_title='Tempo (dias)',
                  yaxis_title='Número de Indivíduos',
                  width=600,
                  height=300,
                  margin=margem
                  )
fig.show()
'''